In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
X, y = fetch_california_housing(as_frame=True, return_X_y=True)

In [11]:
def standard_scaler(X):
    means = X.mean(0)
    stds = X.std(0)
    return (X - means)/stds

def sign(x, first_element_zero = False):
    signs = (-1)**(x < 0)
    if first_element_zero:
        signs[0] = 0
    return signs

In [12]:
class RegularizedRegression:
        
    def _record_info(self, X, y, lam, intercept, standardize):
        
        # standardize 
        if standardize == True: 
            X = standard_scaler(X)
        
        # add intercept
        if intercept == False: 
            ones = np.ones(len(X)).reshape(len(X), 1) # column of ones 
            X = np.concatenate((ones, X), axis = 1) # concatenate
            
        # record values
        self.X = np.array(X)
        self.y = np.array(y)
        self.N, self.D = self.X.shape
        self.lam = lam
        
    def fit_ridge(self, X, y, lam = 0, intercept = False, standardize = True):
        
        # record data and dimensions
        self._record_info(X, y, lam, intercept, standardize)
        
        # estimate parameters
        XtX = np.dot(self.X.T, self.X)
        I_prime = np.eye(self.D)
        I_prime[0,0] = 0 
        XtX_plus_lam_inverse = np.linalg.inv(XtX + self.lam*I_prime)
        Xty = np.dot(self.X.T, self.y)
        self.beta_hats = np.dot(XtX_plus_lam_inverse, Xty)
        
        # get fitted values
        self.y_hat = np.dot(self.X, self.beta_hats)
        
        
    def fit_lasso(self, X, y, lam = 0, n_iters = 2000,
                  lr = 0.0001, intercept = False, standardize = True):

        # record data and dimensions
        self._record_info(X, y, lam, intercept, standardize)
        
        # estimate parameters
        beta_hats = np.random.randn(self.D)
        for i in range(n_iters):
            dL_dbeta = -self.X.T @ (self.y - (self.X @ beta_hats)) + self.lam*sign(beta_hats, True)
            beta_hats -= lr*dL_dbeta 
        self.beta_hats = beta_hats
        
        # get fitted values
        self.y_hat = np.dot(self.X, self.beta_hats)

In [13]:
# set lambda
lam = 10

# fit ridge
ridge_model = RegularizedRegression()
ridge_model.fit_ridge(X, y, lam)

# fit lasso
lasso_model = RegularizedRegression()
lasso_model.fit_lasso(X, y, lam)

C:\Users\DELL\AppData\Local\Temp\ipykernel_7220\2475112960.py:46: RuntimeWarning: overflow encountered in matmul
  dL_dbeta = -self.X.T @ (self.y - (self.X @ beta_hats)) + self.lam*sign(beta_hats, True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_7220\2475112960.py:46: RuntimeWarning: invalid value encountered in matmul
  dL_dbeta = -self.X.T @ (self.y - (self.X @ beta_hats)) + self.lam*sign(beta_hats, True)


## Sklearn implimentation

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### RIDGE

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
alpha = 1

# Ridge
ridge_model = Ridge(alpha = alpha)
ridge_model.fit(X_train, y_train)


# Lasso
lasso_model = Lasso(alpha = alpha)
lasso_model.fit(X_train, y_train);

### Lasso

In [18]:
from sklearn.linear_model import RidgeCV, LassoCV
alphas = [0.01, 1, 100]

# Ridge
ridgeCV_model = RidgeCV(alphas = alphas)
ridgeCV_model.fit(X_train, y_train)

# Lasso
lassoCV_model = LassoCV(alphas = alphas)
lassoCV_model.fit(X_train, y_train);

In [20]:
print('Ridge alpha:', ridgeCV_model.alpha_)
print('Lasso alpha:', lassoCV_model.alpha_)

Ridge alpha: 1.0
Lasso alpha: 0.01


### Elastic Net

In [ ]:
sk_elastic_net_regression = ElasticNet()
sk_elastic_net_regression.fit(X_train, y_train)

sk_elastic_net_pred_res = sk_elastic_net_regression.predict(X_test)
sk_elastic_net_pred_train_res = sk_elastic_net_regression.predict(X_train)

sk_elastic_net_r2 = r2_score(y_test, sk_elastic_net_pred_res)
sk_elastic_net_train_r2 = r2_score(y_train, sk_elastic_net_pred_train_res)

sk_elastic_net_mse = mean_squared_error(y_test, sk_elastic_net_pred_res)
sk_elastic_net_train_mse = mean_squared_error(y_train, sk_elastic_net_pred_train_res)

sk_elastic_net_mape = mean_absolute_percentage_error(y_test, sk_elastic_net_pred_res)
sk_elastic_net_train_mape = mean_absolute_percentage_error(y_train, sk_elastic_net_pred_train_res)

print(f'ElasticNet R2 score: {sk_elastic_net_r2}')
print(f'ElasticNet train R2 score: {sk_elastic_net_train_r2}', '\n')

print(f'ElasticNet MSE: {sk_elastic_net_mse}')
print(f'ElasticNet train MSE: {sk_elastic_net_train_mse}', '\n')

print(f'ElasticNet MAPE: {sk_elastic_net_mape}')
print(f'ElasticNet train MAPE: {sk_elastic_net_train_mape}', '\n')

print(f'prediction: {sk_elastic_net_pred_res}')